
### 📦 1. ANOVA 검정 방법 요약

| **ANOVA 유형**                | **목적**                                                      | **주요 가정**                                                                                                     | **파이썬 구현 (라이브러리/함수)**                                         |
|------------------------------ |-------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------|
| **One-Way ANOVA**             | 독립된 두 개 이상의 그룹 간 평균 차이 검정                    | - 각 그룹 내 정규성<br>- 그룹 간 분산의 동질성<br>- 관측치의 독립성                                               | `scipy.stats.f_oneway`<br>또는 `statsmodels`의 `ols`와 `anova_lm`         |
| **Two-Way ANOVA**             | 두 요인(독립 변수)의 주효과 및 상호작용 효과 검정               | - 각 셀(cell) 내 정규성<br>- 각 셀의 분산 동질성<br>- 관측치의 독립성                                             | `statsmodels.formula.api.ols`와 `sm.stats.anova_lm`                      |
| **Repeated Measures ANOVA**   | 동일 대상에 대해 여러 조건 또는 시간에 따른 측정값의 차이 검정  | - 정규성<br>- 구면성(sphericity: 반복측정 자료에서 조건 간 분산 차이가 없음)<br>- 반복 측정에 의한 종속성           | `statsmodels.stats.anova.AnovaRM`                                        |
| **MANOVA (Multivariate ANOVA)** | 여러 종속 변수에 대해 그룹 간 차이 검정                      | - 다변량 정규성<br>- 공분산 행렬의 동질성<br>- 관측치의 독립성                                                     | `statsmodels.multivariate.manova.MANOVA`                                 |

---

#### 📦 2. 파이썬 코드

##### ✅ 2.1 One-Way ANOVA 예제
세 그룹 간 평균 차이를 검정하는 예제입니다.
```python
import numpy as np
from scipy.stats import f_oneway

# 예시 데이터: 각 그룹별 데이터 (단순 예시)
group1 = np.array([10, 12, 9, 11, 13])
group2 = np.array([14, 15, 13, 16, 14])
group3 = np.array([10, 9, 11, 8, 10])

# One-Way ANOVA 수행
F_stat, p_value = f_oneway(group1, group2, group3)
print("One-Way ANOVA 결과:")
print("F-statistic:", F_stat)
print("p-value:", p_value)

# Tukey의 HSD 사후 검정 수행
from statsmodels.stats.multicomp import pairwise_tukeyhsd
tukey_result = pairwise_tukeyhsd(endog=df['score'],
                                 groups=df['group'],
                                 alpha=0.05)
print(tukey_result)

# 결과 시각화: 각 그룹 간 평균 차이와 신뢰구간
tukey_result.plot_simultaneous(comparison_name='A')  # 기준 그룹을 선택할 수 있음
plt.title("Tukey HSD: Group Comparisons")
plt.xlabel("Mean Difference")
plt.show()
```
--- 
##### ✅ 2.2 Two-Way ANOVA
```python
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

# 예시 데이터 생성 (두 요인: factor1, factor2)
data = {
    'dependent': [23, 21, 25, 30, 22, 19, 27, 35, 20, 18, 26, 28],
    'factor1': ['A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C'],
    'factor2': ['X', 'X', 'Y', 'Y', 'X', 'X', 'Y', 'Y', 'X', 'X', 'Y', 'Y']
}
df = pd.DataFrame(data)

# OLS 모델 적합 및 ANOVA 테이블 생성 (Type II)
model = ols('dependent ~ C(factor1) + C(factor2) + C(factor1):C(factor2)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print("Two-Way ANOVA 결과:")
print(anova_table)

# Type II ANOVA 테이블 생성 
> 주효과를 다른 주효과에 대해 평가하며, 상호작용은 주 효과 평가 시 고려하지 않음 
anova_table_type2 = sm.stats.anova_lm(model, typ=2)
print("ANOVA Table (Type II):")
print(anova_table_type2)

# Type III ANOVA 테이블 생성
> 각 효과(주효과와 상호작용 효과 모두)를 모든 다른 효과에 대해 평가
anova_table_type3 = sm.stats.anova_lm(model, typ=3)
print("\nANOVA Table (Type III):")
print(anova_table_type3)

```
--- 
##### ✅ 2.3 Repeated Measures ANOVA
```python
from statsmodels.stats.anova import AnovaRM

# 예시 데이터 생성: 각 subject가 여러 조건(condition)에서 측정됨
df_rm = pd.DataFrame({
    'subject': [1, 1, 1, 2, 2, 2, 3, 3, 3],
    'condition': ['A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C'],
    'score': [23, 21, 25, 30, 22, 19, 27, 35, 20]
})

aovrm = AnovaRM(df_rm, 'score', 'subject', within=['condition'])
rm_results = aovrm.fit()
print("Repeated Measures ANOVA 결과:")
print(rm_results)
``` 
##### ✅ 2.4 MANOVA - 여러 종속 변수에 대해 그룹 간 차이를 검정하는 예제
```python
import pandas as pd
from statsmodels.multivariate.manova import MANOVA

# 예시 데이터 생성: 그룹별로 두 개의 종속 변수 측정
data = {
    'group': ['A', 'A', 'B', 'B', 'C', 'C'],
    'y1': [20, 21, 23, 22, 25, 27],
    'y2': [30, 29, 28, 30, 27, 26]
}
df_manova = pd.DataFrame(data)

maov = MANOVA.from_formula('y1 + y2 ~ group', data=df_manova)
manova_results = maov.mv_test()
print("MANOVA 결과:")
print(manova_results)


```

#### 📦 사후 분석(Post-Hoc Analysis) 정리

| **분석 상황**                  | **조건**                                                       | **추천 사후 분석 방법**      | **비고**                                         |
|--------------------------------|----------------------------------------------------------------|-----------------------------|--------------------------------------------------|
| **모수적 ANOVA**               | 정규성 및 등분산성(분산 동질성) 모두 만족                        | Tukey's HSD                | 그룹 간 평균 차이를 비교 (One-Way ANOVA 후)        |
| **Welch's ANOVA**              | 정규성은 만족하지만 **등분산성 위배**                           | Games-Howell Test          | 분산 이질성을 고려한 다중 비교                    |
| **비모수적 ANOVA (Kruskal-Wallis)** | 정규성이 위배된 경우 (등분산성 여부와 관계없이)                | Dunn's Test                | 중위수 비교 – 보정법(Bonferroni, Holm 등) 적용 가능  |

---

##### ✅ 1. Tukey's HSD (One-Way ANOVA 후)

```python
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# 예시 데이터: 그룹별 측정값 (score)
data = {
    'score': [23, 21, 25, 30, 22, 19, 27, 35, 20, 18, 26, 28],
    'group': ['A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C']
}
df = pd.DataFrame(data)

# Tukey HSD 사후 검정
tukey_result = pairwise_tukeyhsd(endog=df['score'],
                                 groups=df['group'],
                                 alpha=0.05)
print(tukey_result)

# 결과 시각화: 각 그룹 간 평균 차이와 신뢰구간
tukey_result.plot_simultaneous(comparison_name='A')
plt.title("Tukey HSD: Group Comparisons")
plt.xlabel("Mean Difference")
plt.show()


##### ✅ 2. Games-Howell Test (Welch's ANOVA 후)
```python
import pandas as pd
import pingouin as pg
# 예시 데이터: 그룹 간 score (등분산성이 위배된 상황)
data = {
    'score': [23, 20, 25, 30, 32, 29, 27, 35, 24, 28, 26, 31],
    'group': ['A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C']
}
df = pd.DataFrame(data)

# Games-Howell 사후 검정 수행
gh_results = pg.pairwise_gameshowell(data=df, dv='score', between='group')
print(gh_results)


##### ✅ 3. Dunn's Test (Kruskal-Wallis 후)
```python
import pandas as pd
import scikit_posthocs as sp

# 예시 데이터: 그룹 간 score (정규성 위배 상황)
data = {
    'score': [15, 18, 16, 19, 22, 20, 25, 23, 17, 21, 18, 24],
    'group': ['A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C']
}
df = pd.DataFrame(data)

# Dunn's 사후 검정 (p-value 보정: Bonferroni)
dunn_results = sp.posthoc_dunn(df, val_col='score', group_col='group', p_adjust='bonferroni')
print(dunn_results)

## ✅ 비모수적 ANOVA 요약표

| **분석 상황**                   | **비모수적 방법**           | **설명**                                                         | **파이썬 함수 / 라이브러리**                       |
|----------------------------------|-----------------------------|------------------------------------------------------------------|--------------------------------------------------|
| **독립 표본 비교** (1요인)       | `Kruskal-Wallis Test`       | 세 그룹 이상 간 **중위수** 차이 비교                                | `scipy.stats.kruskal`                            |
| **관련 표본 / 반복측정 비교**   | `Friedman Test`             | 같은 대상에 대한 **반복 조건 간 중위수** 비교                       | `scipy.stats.friedmanchisquare`                 |
| **두 요인 이상 비교** (2-way 등) | `Scheirer-Ray-Hare Test`    | **2-way 비모수 ANOVA** / 상호작용 포함 가능                        | 직접 구현 또는 `R` 통계 패키지 사용 권장           |
| **Kruskal 이후 사후분석**        | `Dunn's Test`               | Kruskal 결과 유의할 때 **쌍별 비교**                                | `scikit-posthocs.posthoc_dunn`                  |
| **Friedman 이후 사후분석**       | `Nemenyi Test`              | Friedman 결과 유의할 때 **쌍별 비교**                                | `scikit-posthocs.posthoc_nemenyi_friedman`      |

### ✅ Friedman Test 

```python
import pandas as pd
from scipy.stats import friedmanchisquare

# 예시 데이터: 같은 피험자에서 세 조건을 측정한 값
data = {
    'subject': [1, 2, 3, 4, 5],
    'condition1': [5, 6, 7, 8, 7],
    'condition2': [6, 7, 6, 7, 8],
    'condition3': [7, 8, 7, 6, 7]
}
df = pd.DataFrame(data)

# Friedman 테스트 수행 (세 조건 비교)
stat, p = friedmanchisquare(df['condition1'], df['condition2'], df['condition3'])
print("Friedman Test 통계량: ", stat)
print("p-value: ", p)


### ✅  Kruskal-Wallis Test

```python
import numpy as np
import pandas as pd
from scipy.stats import kruskal

# 예시 데이터: 3그룹의 관측값 (모수적 가정 미충족 상황을 가정)
group1 = [12, 15, 14, 10, 13]
group2 = [22, 25, 20, 19, 24]
group3 = [30, 29, 35, 33, 31]

# Kruskal-Wallis Test 수행
stat, p = kruskal(group1, group2, group3)
print("Kruskal-Wallis Test 결과:")
print("통계량 H =", stat)
print("p-value =", p)

# 해석:
# p-value가 0.05보다 작으면, 그룹 간에 유의한 중위수 차이가 있다고 결론 내림.